In [15]:

import importlib
import re
import sys
import time

import tools
import sunfish
import chess
import random
import chess.engine
import chess.svg

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import tensorflow as tf 
import pandas as pd
import numpy as np



from tools import WHITE, BLACK
def convertToFeatureVector(boardState):
    pieces = []
    encoding = {'P':1,'R':2,'N':3,'B':4,'Q':5,'K':6,'p':-1,'r':-2,'n':-3,'b':-4,'q':-5,'k':-6, '.':0}
    for i in range(64):
        piece = boardState.piece_at(i)
        if piece:
            piece = piece.symbol()
        else:
            piece = '.'
        pieces.append(encoding[piece])
    return pieces

print("Loading Model")
model = tf.keras.models.load_model("deepchess.h5")
print("Done Loading")
def convertBoardToPos():
    
def evaluateMove(move1, move2):
    #simple_dense.h5
    
    df1 = pd.DataFrame(data=[move1])
    df2 = pd.DataFrame(data=[move2])
    return model.predict([df1, df2])

def countMoves(moves):
    length = 0
    for _ in moves:
        length += 1
    return length

def generateMove(board):
        ogBoard = board
        count = 0
        move1 = move2 = None
        board1 = board2 = None

        for move in ogBoard.legal_moves:
            if count == 0:
                move1 = move
            else:
                modBoard = ogBoard
                move2 = move
                
                modBoard.push(move1)
                move1Feature = convertToFeatureVector(modBoard)
                modBoard.pop()
                
                modBoard.push(move2)
                move2Feature = convertToFeatureVector(modBoard)
                modBoard.pop()
                strengthArr = evaluateMove(move1Feature, move2Feature)[0]
                print(move1, move2, strengthArr)
                if(strengthArr[0] < strengthArr[1]):
                    move1 = move2
                #compare
            count += 1
        return move1.uci()
    

# Disable buffering
class Unbuffered(object):
    def __init__(self, stream):
        self.stream = stream
    def write(self, data):
        self.stream.write(data)
        self.stream.flush()
        sys.stderr.write(data)
        sys.stderr.flush()
    def __getattr__(self, attr):
        return getattr(self.stream, attr)
sys.stdout = Unbuffered(sys.stdout)

from datetime import datetime
now = datetime.now()
path = 'sunfish-' + now.strftime("%d:%m:%Y-%H:%M:%S:%f") + '.log'
sys.stderr = open(path, 'a')

def main():
    pos = tools.parseFEN(tools.FEN_INITIAL)
    
    searcher = sunfish.Searcher()
    forced = False
    color = WHITE
    our_time, opp_time = 1000, 1000 # time in centi-seconds
    show_thinking = False
    options = {}
    history = []

    stack = []
    count = 0
    while True:
        if stack:
            smove = stack.pop()
        else:
            smove = generateMove()
        
        if smove == 'quit':
            break
        
        elif smove == 'protover 2':
            print('feature done=0')
            print('feature myname="Sunfish"')
            print('feature usermove=1')
            print('feature setboard=1')
            print('feature ping=1')
            print('feature sigint=0')
            print('feature variants="normal"')
            print('feature option="qs_limit -spin {} -100 1000"'.format(sunfish.QS_LIMIT))
            print('feature option="eval_roughness -spin {} 1 1000"'.format(sunfish.EVAL_ROUGHNESS))
            print('feature option="draw_test -spin {} 0 1"'.format(int(sunfish.DRAW_TEST)))
            print('feature done=1')

        elif smove == 'new':
            stack.append('setboard ' + tools.FEN_INITIAL)
            # Clear out the old searcher, including the tables
            searcher = sunfish.Searcher()
            del history[:]

        elif smove.startswith('setboard'):
            _, fen = smove.split(' ', 1)
            pos = tools.parseFEN(fen)
            color = WHITE if fen.split()[1] == 'w' else BLACK
            del history[:]

        elif smove == 'force':
            forced = True

        elif smove.startswith('option'):
            _, aeqb = smove.split(maxsplit=1)
            if '=' in aeqb:
                name, val = aeqb.split('=')
            else: name, val = aeqb, True
            if name == 'qs_limit':
                sunfish.QS_LIMIT = int(val)
            if name == 'eval_roughness':
                sunfish.EVAL_ROUGHNESS = int(val)
            if name == 'draw_test':
                sunfish.DRAW_TEST = bool(int(val))
            options[name] = val

        elif smove == 'go':
            forced = False

            moves_remain = 40
            use = our_time/moves_remain
            # Let's follow the clock of our opponent
            if our_time >= 100 and opp_time >= 100:
                use *= our_time/opp_time

            start = time.time()
            for ply, move, score in searcher.search(pos, history):
                entry = searcher.tp_score.get((pos, ply, True))
                score = int(round((entry.lower + entry.upper)/2))
                if show_thinking:
                    used = int((time.time() - start)*100 + .5)
                    moves = tools.pv(searcher, pos, include_scores=False)
                    seldepth = 0
                    nps = int(searcher.nodes / (time.time() - start) + .5)
                    print('{:>3} {:>8} {:>8} {:>13} {:>1} {:>4} \t{}'.format(
                        ply, score, used, searcher.nodes, seldepth, nps, moves))
                    print('# Hashfull: {:.3f}%; {} <= score < {}'.format(
                        len(searcher.tp_score)/sunfish.TABLE_SIZE*100, entry.lower, entry.upper))
                # If found mate, just stop
                if entry.lower >= sunfish.MATE_UPPER:
                    break
                if time.time() - start > use/100:
                    break
            # We sometimes make illegal moves when we're losing,
            # so it's safer to just resign.
            if score == -sunfish.MATE_UPPER:
                print('resign')
            else:
                print('move1', tools.mrender(pos, move))
            pos = pos.move(move)
            history.append(pos)
            color = 1-color

        elif smove.startswith('ping'):
            _, N = smove.split()
            print('pong', N)

        elif smove.startswith('usermove'):
            _, smove = smove.split()
            m = tools.mparse(color, smove)
            pos = pos.move(m)
            history.append(pos)
            print(history)
            break
            color = 1-color
            if not forced:
                stack.append('go')

        elif smove.startswith('time'):
            our_time = int(smove.split()[1])

        elif smove.startswith('otim'):
            opp_time = int(smove.split()[1])

        elif smove.startswith('perft'):
            start = time.time()
            for d in range(1,10):
                res = sum(1 for _ in tools.collect_tree_depth(tools.expand_position(pos), d))
                print('{:>8} {:>8}'.format(res, time.time()-start))

        elif smove.startswith('post'):
            show_thinking = True

        elif smove.startswith('nopost'):
            show_thinking = False

        elif any(smove.startswith(x) for x in ('xboard','random','hard','accepted','level','easy','st','result','?')):
            print('# Ignoring command {}.'.format(smove))

        elif smove.startswith('reject'):
            _, feature = smove.split()[:2] # split(maxsplit=2) doesnt work in python2.7
            print('# Warning ({} rejected): Might not work as expected.'.format(feature))

        else:
            print('# Warning (unkown command): {}. Treating as move.'.format(smove))
            stack.append('usermove {}'.format(smove))
        count += 1
        #print(count, smove)

if __name__ == '__main__':
    main()


Loading Model
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Done Loading


TypeError: generateMove() missing 1 required positional argument: 'board'